In [16]:
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from preprocessing.data_prep import preprocess
import cairosvg
from xml.dom import minidom
from svg.path import parse_path
import math

from statistics import mean, stdev 

ALPHA = 0.95
BETA = 0.05
SPAWN_COUNT = 5

basefiles = preprocess('/Users/anjueappen/svg')
   
def compute_prob(string, index):
    order = index
    try:
        length = parse_path(string).length()
    except ZeroDivisionError:
        length = 0
    return math.exp(ALPHA*order)/math.exp(BETA*length)
    
for basefile, _ in basefiles[:1]:
    doc = minidom.parse(basefile)
    path_strings = [(path, compute_prob(path.getAttribute('d'), index)) for index, path
                    in enumerate(doc.getElementsByTagName('path'))]
    
    vals = [item[1] for item in path_strings]
    avg = mean(vals)
#     std = stdev(vals)
    
    path_strings = [(string, prob/avg) for string, prob in path_strings]
    
#     import pprint;pprint.pprint(path_strings)
    for i in range(SPAWN_COUNT):
        path_strings.remove(max(path_strings, key=lambda x: x[1])) 
        print(len(path_strings))
        doc.childNodes[1].childNodes[1].childNodes[1].childNodes = [string[0] for string in path_strings]
        
        spawn_file = basefile.replace('svg', 'png').replace('.png', '_%d.png' % i)
        svg_code = doc.toxml()
        
        with open(spawn_file,'w') as fout:
            cairosvg.svg2png(bytestring=svg_code, write_to=fout)
        print spawn_file
        
    doc.unlink()
    
    
# fout = open('output.png','w')

# svg_code = """<?xml version="1.0" standalone="no"?>
# <!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" 
#   "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
# <svg width="5cm" height="4cm" version="1.1"
#      xmlns="http://www.w3.org/2000/svg">
#   <desc>Four separate rectangles
#   </desc>
#     <rect x="0.5cm" y="0.5cm" width="2cm" height="1cm"/>
#     <rect x="0.5cm" y="2cm" width="1cm" height="1.5cm"/>
#     <rect x="3cm" y="0.5cm" width="1.5cm" height="2cm"/>
#     <rect x="3.5cm" y="3cm" width="1cm" height="0.5cm"/>

#   <!-- Show outline of canvas using 'rect' element -->
#   <rect x=".01cm" y=".01cm" width="4.98cm" height="3.98cm"
#         fill="none" stroke="blue" stroke-width=".02cm" />

# </svg>
# """

# cairosvg.svg2png(bytestring=svg_code,write_to=fout)

# fout.close()



18
/Users/anjueappen/png/airplane/1_0.png
17
/Users/anjueappen/png/airplane/1_1.png
16
/Users/anjueappen/png/airplane/1_2.png
15
/Users/anjueappen/png/airplane/1_3.png
14
/Users/anjueappen/png/airplane/1_4.png
